# 라이브러리

In [1]:
import os
import torch
from datasets import load_dataset
import transformers
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import nltk
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from torch.utils.data import TensorDataset, DataLoader, Subset
from tensorflow.keras.utils import pad_sequences

from torch.utils.data import TensorDataset, DataLoader, Subset
from tqdm import tqdm



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tqdm.pandas()

# HuggingFace Model, DataLoader

In [2]:
from transformers import BertTokenizer, BertModel, DistilBertTokenizer, DistilBertModel

model = DistilBertModel.from_pretrained("lvwerra/distilbert-imdb")
tokenizer = DistilBertTokenizer.from_pretrained("lvwerra/distilbert-imdb")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin d:\anaconda\envs\Pray\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda122.dll
CUDA SETUP: CUDA runtime path found: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.2\bin\cudart64_12.dll
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 122
CUDA SETUP: Loading binary d:\anaconda\envs\Pray\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda122.dll...


d:\anaconda\envs\Pray\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {WindowsPath('D:/anaconda/envs/Pray/bin')}
  warn(msg)
d:\anaconda\envs\Pray\Lib\site-packages\bitsandbytes\cuda_setup\main.py:156: UserWarning: D:\anaconda\envs\Pray did not contain ['cudart64_110.dll', 'cudart64_120.dll', 'cudart64_12.dll'] as expected! Searching further paths...
  warn(msg)


In [3]:
# Download in https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
df1 = pd.read_csv("IMDB Dataset.csv")
df1.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
def make_sentiment(row):
    if row['sentiment'] == 'positive':
        row['sentiment'] =  1
    else:
        row['sentiment'] = 0
        
    return row

In [5]:
df1 = df1.progress_apply(make_sentiment, axis = 1)

100%|██████████| 50000/50000 [00:03<00:00, 15296.65it/s]


In [6]:
train_all_idx, test_idx = train_test_split(df1.index,test_size = 0.2,
                                       random_state = 42, stratify = df1['sentiment'])
# 8:2 로 train / test split

train_idx, val_idx = train_test_split(df1.iloc[train_all_idx].index,test_size = 0.25, random_state = 42, stratify = df1.iloc[train_all_idx]['sentiment'])
# 6 : 2 : 2 로 train / valid / test

In [7]:
df_train = df1.iloc[train_idx].copy()
df_test = df1.iloc[test_idx].copy()
df_val = df1.iloc[val_idx].copy()

In [8]:
df_train['sentiment'].value_counts()

sentiment
1    15000
0    15000
Name: count, dtype: int64

In [9]:
df_test['sentiment'].value_counts()

sentiment
0    5000
1    5000
Name: count, dtype: int64

In [10]:
df_val['sentiment'].value_counts()

sentiment
1    5000
0    5000
Name: count, dtype: int64

In [11]:
# ff= np.array([len(i) for i in tokenizer.batch_encode_plus( df1['review'].values)['input_ids']])

In [12]:
# import matplotlib.pyplot as plt

# plt.hist(ff)

# np.quantile(ff,0.8)

In [13]:
# max_len = 512

# train_sentence = tokenizer([i for i  in df_train['review'].values], add_special_tokens=True, 
#                                              padding='max_length', truncation=True, max_length=max_len)


In [14]:
# val_sentence = tokenizer([i for i in list(df_val['review'].values)], add_special_tokens=True, 
#                                              padding='max_length', truncation=True, max_length=max_len)

In [15]:
# test_sentence = tokenizer([i for i in list(df_test['review'].values)], add_special_tokens=True, 
#                                             padding='max_length', truncation=True, max_length=max_len)

In [16]:
# df2 = df1.reset_index(drop = True).copy()

In [17]:
# train_sentence['label'] = df2.iloc[train_idx].reset_index(drop = True)['sentiment']
# val_sentence['label'] = df2.iloc[val_idx].reset_index(drop = True)['sentiment']
# test_sentence['label'] = df2.iloc[test_idx].reset_index(drop = True)['sentiment']

In [18]:
# # Save
# import pickle

# with open('train_sentence.pickle', 'wb') as f:
#     pickle.dump(train_sentence, f, pickle.HIGHEST_PROTOCOL)
 
# with open('val_sentence.pickle', 'wb') as f:
#     pickle.dump(val_sentence, f, pickle.HIGHEST_PROTOCOL)

# with open('test_sentence.pickle', 'wb') as f:
#     pickle.dump(test_sentence, f, pickle.HIGHEST_PROTOCOL)

In [19]:
# load
import pickle


with open('train_sentence.pickle', 'rb') as f:
    train_sentence = pickle.load(f)
    
# load
with open('val_sentence.pickle', 'rb') as f:
    val_sentence = pickle.load(f)
    
# load
with open('test_sentence.pickle', 'rb') as f:
    test_sentence = pickle.load(f)

In [20]:
class EmoDataset(Dataset):
    def __init__(self, encoded_data):
        self.input_ids = encoded_data['input_ids']
        # self.token_type_ids = encoded_data['token_type_ids']
        self.attention_mask = encoded_data['attention_mask']
        self.label = encoded_data['label']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
            # 'token_type_ids': torch.tensor(self.token_type_ids[idx], dtype=torch.long),
            'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
            'label': torch.tensor(self.label[idx], dtype=torch.float)
            }

In [21]:
train_dataset = EmoDataset(train_sentence)
test_dataset = EmoDataset(test_sentence)
val_dataset = EmoDataset(val_sentence)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,drop_last=True)  # batch_size는 원하는 값으로 설정하세요.
valid_loader = DataLoader(val_dataset, batch_size=64, shuffle=True,drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True,drop_last=True)

# Model Train (Fine-tunning)

In [22]:
class SentimentClassifier(nn.Module):
    def __init__(self, model, num_labels=1):
        super(SentimentClassifier, self).__init__()
        self.model = model
        self.dropout = nn.Dropout(p=0.1)
        self.dropout_2 = nn.Dropout(0.3)
        # self.classifier = nn.Linear(768, 3)
        # self.classifier_1 = nn.Linear(768, 512)
        # self.classifier_2 = nn.Linear(512, 128)
        self.classifier_3 = nn.Linear(768, num_labels) # 768은 BERT의 hidden size, num_labels는 분류하려는 클래스 수 (긍정/부정이므로 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids, attention_mask)

        pooled_output = torch.mean(outputs['last_hidden_state'],dim = 1)
        
        
        output_3 = self.dropout(pooled_output)
        logits = self.classifier_3(output_3)

        return logits
    
# 모든 매개변수를 리스트로 변환

sentiment_model = SentimentClassifier(model).to(device)

# 1. 모든 파라미터를 freeze
for param in sentiment_model.parameters():
    param.requires_grad = False

# 2. 분류 레이어의 파라미터만 학습되도록 설정

for param in sentiment_model.classifier_3.parameters():
    param.requires_grad = True
    

# # 3. 마지막 attention 레이어의 파라미터만 학습되도록 설정
# for param in sentiment_model.model.encoder.layer[11].parameters():
#     param.requires_grad = True

# # 3. 마지막 attention 레이어의 파라미터만 학습되도록 설정
# for param in sentiment_model.kobert.encoder.layer[10].parameters():
#     param.requires_grad = True

In [23]:
sentiment_model

SentimentClassifier(
  (model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [24]:
# for batch in tqdm(train_loader, desc="Processing", leave =False):
#     input_ids = batch['input_ids'].to(device)
#     # token_type_ids = batch['token_type_ids'].to(device)
#     attention_mask = batch['attention_mask'].to(device)
#     train_y =  batch['label'].to(device).float()


#     outputs = sentiment_model(input_ids=input_ids, attention_mask = attention_mask)
#     break

In [25]:
from copy import deepcopy
import time

save_path = 'DistrilBERT_IMDB.pth'
def train_model(model, criterion, optimizer, early_stop, epochs, train_loader, valid_loader):
    train_losses, train_accuracies, valid_losses, valid_accuracies, lowest_loss, lowest_epoch = list(), list(), list(), list(), np.inf, 0

    # DEBUG
    # progress_count = 0

    for epoch in range(epochs):
        train_loss, train_accuracy, train_corrects, valid_loss, valid_accuracy, valid_corrects = 0, 0, 0, 0, 0, 0
        train_correct, valid_correct = 0, 0

        start = time.time()

        model.train()
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to(device)
            # token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            train_y =  batch['label'].to(device).float()


            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            
            optimizer.zero_grad()            
            
            loss = criterion(outputs.reshape(-1), train_y)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            pred = torch.round(torch.sigmoid(outputs.detach().clone().reshape(-1)))
            train_correct += pred.eq(train_y.detach().clone()).sum().item()


            # DEBUG
            # if (progress_count % 10) == 0:
            #    print (y_pred.eq(train_y.detach().cpu()).sum().item(), len(y_pred))
            # progress_count += 1


        train_loss = train_loss / len(train_loader)
        train_losses.append(train_loss)
        train_accuracy = train_correct / len(train_loader.dataset)
        
        model.eval()
        with torch.no_grad():
            for batch in tqdm(valid_loader):
                input_ids = batch['input_ids'].to(device)
                # token_type_ids = batch['token_type_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                valid_y = batch['label'].to(device).float()
                outputs = model(input_ids = input_ids, attention_mask = attention_mask)

                loss = criterion(outputs.reshape(-1), valid_y)
                valid_loss += loss.item()
                pred = torch.round(torch.sigmoid(outputs.detach().clone().reshape(-1)))
                valid_correct += pred.eq(valid_y.detach().clone()).sum().item()

        valid_loss = valid_loss / len(valid_loader)
        valid_losses.append(valid_loss)
        valid_accuracy = valid_correct / len(valid_loader.dataset)
        valid_accuracies.append(valid_accuracy)

        elapsed_time = time.time() - start
        print(f"[Epoch {epoch+1}/{epochs}]: {elapsed_time:.3f} sec(elapsed time), train loss: {train_losses[-1]:.4f}, valid loss: {valid_losses[-1]:.4f}, train acc: {train_accuracy * 100:.3f}%, valid acc: {valid_accuracy * 100:.3f}%")


        if valid_losses[-1] < lowest_loss:
            lowest_loss = valid_losses[-1]
            lowest_epoch = epoch
            best_model = deepcopy(model.state_dict())
            torch.save(best_model, save_path)
        else:
            if (early_stop > 0) and lowest_epoch + early_stop < epoch:
                print ("Early Stopped", epoch, "epochs")
                break


        wandb.log({"Training loss" : train_losses[-1], "Validation loss" : valid_losses[-1] ,"Lowest loss": lowest_loss,
                   'Lowest_epoch' : lowest_epoch, 'epoch' : epoch, 
                   "Valid_Accuracy" : 100 *valid_accuracy,
                   })


    model.load_state_dict(best_model)
    return model, lowest_loss, train_losses, valid_losses, train_accuracies, valid_accuracies

In [26]:
import time
import wandb

criterion = nn.BCEWithLogitsLoss()
sentiment_model.to(device)
optimizer =torch.optim.Adam(sentiment_model.parameters())
early_stop = 10
epochs = 100

In [27]:
wandb.init(
    # set the wandb project where this run will be logged
    project="distilbert-imdb",

    # track hyperparameters and run metadata
    config={
    "learning_rate": None,
    "architecture": "distilbert",
    "optimizer" : 'Adam',
    'loss' : 'BCEWithLogitsLoss'
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yoon303b (ku_software). Use `wandb login --relogin` to force relogin


In [28]:
sentiment_model_best, lowest_loss, train_losses, valid_losses, train_accuracies, valid_accuracies =  train_model(
    sentiment_model, criterion, optimizer, early_stop, epochs, train_loader, valid_loader)

100%|██████████| 156/156 [01:22<00:00,  1.88it/s]


[Epoch 1/100]: 348.289 sec(elapsed time), train loss: 0.1756, valid loss: 0.1527, train acc: 93.543%, valid acc: 94.420%


 15%|█▍        | 68/468 [00:39<03:52,  1.72it/s]


KeyboardInterrupt: 

In [28]:
sentiment_model.load_state_dict(torch.load('DistrilBERT_IMDB.pth'))

<All keys matched successfully>

In [30]:
test_loss = 0
test_correct = 0
test_loss = 0

sentiment_model.eval()
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        # token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        test_y = batch['label'].to(device).float()
        pred = sentiment_model(input_ids,  attention_mask=attention_mask)

        loss = criterion(pred.reshape(-1), test_y)
        test_loss += loss.item()
        y_pred = torch.round(torch.sigmoid(pred.detach().clone().reshape(-1)))
        test_correct += y_pred.eq(test_y.detach()).sum().item()

        
        
    test_loss = test_loss / len(test_loader)
    test_accuracy = test_correct / len(test_loader.dataset)

100%|██████████| 156/156 [01:25<00:00,  1.83it/s]


In [31]:
test_correct

9437

In [32]:
test_accuracy

0.9437

# 매우 큰 과적합